In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import ipywidgets as widgets
from IPython.display import display
import importlib

from PIL import Image

: 

## Import linii 

In [21]:
file_paths = {
    "vertical_horizontal": {
        "resource": "resources/lines/wagon/low_10_n.json",
        "image": "resources/lines/wagon/wagon.png"
    },
    "szyny": {
        "resource": "resources/lines/szyny/szyny.json",
        "image": "resources/lines/szyny/szyny.png"
    }
}

file_path = file_paths["vertical_horizontal"]["resource"]
with open(file_path, "r") as file:
    vertical_horizontal = json.load(file)

file_path = file_paths["szyny"]["resource"]
with open(file_path, "r") as file:
    distances = json.load(file)


print(vertical_horizontal[0]["lines"])
print(distances[0]["lines"])

[[[5, 125], [34, 115], [75, 102], [111, 95], [151, 87], [194, 73], [230, 65], [272, 59], [311, 51], [353, 47]], [[614, 25], [1903, 287], [728, 27], [1864, 268], [917, 34], [1763, 222], [1134, 59], [1637, 175], [1335, 95], [1476, 125]], [[465, 647], [1905, 714], [611, 668], [1756, 720], [795, 684], [1626, 723], [981, 696], [1500, 718], [1153, 711], [1357, 716]], [[509, 599], [1896, 682], [627, 615], [1803, 683], [759, 637], [1669, 680], [925, 649], [1506, 679], [1129, 662], [1348, 678]], [[11, 774], [1901, 887], [79, 803], [1768, 917], [246, 859], [1525, 960], [477, 916], [1283, 974], [700, 948], [976, 968]], [[13, 732], [1901, 851], [1629, 897], [190, 792], [470, 855], [921, 916], [1248, 925], [1439, 913], [1772, 881], [659, 887]], [[1594, 849], [1474, 859], [1367, 859], [587, 823], [489, 803], [385, 782], [295, 762], [887, 843], [1119, 851], [1272, 856]]]
[[[21, 742], [74, 760], [131, 782], [197, 803], [279, 827], [364, 846], [462, 868], [584, 889], [695, 908], [832, 918], [964, 936],

In [22]:
def check_if_all_equal(list):
    if len(list) < 2:
        return True
    return all(ele == list[0] for ele in list)

horizontal_line_length = [len(i) for i in vertical_horizontal[0]["lines"]] 
vertical_line_length = [len(i) for i in vertical_horizontal[1]["lines"]]
print("horizontal", horizontal_line_length)
print("vertical", vertical_line_length)

horizontal_points = np.array(vertical_horizontal[0]["lines"], dtype=np.float64)
vertical_points = np.array(vertical_horizontal[1]["lines"], dtype=np.float64)

distances_array = np.array(distances[0]["lines"], dtype=np.float64)

print("(liczba linii, liczba punktów w linii, współrzędne x i y)")
print(f"horizontal: {horizontal_points.shape}, vertical: {vertical_points.shape}")

horizontal [10, 10, 10, 10, 10, 10, 10]
vertical [10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
(liczba linii, liczba punktów w linii, współrzędne x i y)
horizontal: (7, 10, 2), vertical: (14, 10, 2)


## Wyświetlanie punktów na obrazie 

In [23]:
from enum import Enum
from dataclasses import dataclass


def scale_image_by_percent(image, percent):
    width = int(image.shape[1] * percent / 100)
    height = int(image.shape[0] * percent / 100)
    dim = (width, height)

    return cv2.resize(image, dim, interpolation=cv2.INTER_AREA)


class DotStyles(Enum):
    original_horizontal = {
        "radius": 5,
        "thickness": -1,
        "color": (255, 0, 0)
    }
    original_vertical = {
        "radius": 5,
        "thickness": -1,
        "color": (255, 125, 125)
    }
    rectified_horizontal = {
        "radius": 5,
        "thickness": -1,
        "color": (0, 0, 255)
    }
    rectified_vertical = {
        "radius": 5,
        "thickness": -1,
        "color": (125, 125, 255)
    }


@dataclass
class DotDisplay:
    lines: np.ndarray
    style: DotStyles


def display_image_with_points(image, window_name, display_scale=70, *dot_displays):
    img = image.copy()

    for dot_display in dot_displays:
        for line in dot_display.lines.astype(int):
            for point in line:
                cv2.circle(img, center=point, **dot_display.style.value)

    cv2.imshow(window_name, scale_image_by_percent(img, display_scale))
    cv2.waitKey(0)

In [24]:
horizontal_display = DotDisplay(horizontal_points, DotStyles.original_horizontal)
vertical_display = DotDisplay(vertical_points, DotStyles.original_vertical)
distances_display =  DotDisplay(distances_array, DotStyles.original_vertical)

std_button = widgets.Button(description = "show std image")
distance_button = widgets.Button(description = "show distance image")

display(std_button, distance_button)

def std_button_clicked(b):
    image = cv2.imread(file_paths["vertical_horizontal"]["image"])
    display_image_with_points(image, "original", 70, horizontal_display, vertical_display)
    
def distance_button_clicked(b):
    image = cv2.imread(file_paths["szyny"]["image"])
    display_image_with_points(image, "original", 70, distances_display)
    
    
std_button.on_click(std_button_clicked)
distance_button.on_click(distance_button_clicked)

Button(description='show std image', style=ButtonStyle())

Button(description='show distance image', style=ButtonStyle())

## Optymalizacja 

In [25]:
from itertools import tee


def pairwise(iterable):
    # pairwise('ABCDEFG') --> AB BC CD DE EF FG
    a, b = tee(iterable)
    next(b, None)
    return list(zip(a, b))

def middle_value(array):
    length = array.shape[1]

    if length % 2 == 0:
        upper_index = length // 2 
        lower_index = lower_index - 1
        return (array[:, lower_index] + array[:, upper_index]) / 2
    else:
        middle_index = length // 2
        return array[:, middle_index]
    
def deviation(array, mean_array):
    x = abs(array - mean_array)**2
    return np.sqrt(np.mean(x, 1))

def calc_distance(array):
    x = array[:, :, 0]
    x = np.apply_along_axis(pairwise, 1, x)
    x = np.abs(x[:, :, 0] - x[:, :, 1])

    # middle = middle_value(x)
    # return deviation(x, middle.reshape(len(middle), 1)).sum()
    return x.std(axis=1).sum()

In [26]:
from scipy.optimize import minimize
import math
import cv2

# global
sensor_h = 3.69
sensor_w = 7.38
W = 1920
H = 1080

weights = {
    "std": 0.7,
    "distance": 1
}

def rectify_points(config, points):
    X = config[0]
    Y = config[1]
    alpha = config[2]
    beta = config[3]
    gamma = config[4]
    focus = config[5]
    scale = config[6]
    dist = config[7:]

    Rx = np.array([[1, 0, 0], [0, math.cos(alpha), -math.sin(alpha)], [0, math.sin(alpha), math.cos(alpha)]])
    Ry = np.array([[math.cos(beta), 0, -math.sin(beta)], [0, 1, 0], [math.sin(beta), 0, math.cos(beta)]])
    Rz = np.array([[math.cos(gamma), -math.sin(gamma), 0], [math.sin(gamma), math.cos(gamma), 0], [0, 0, 1]])
    R = np.matmul(Rx, np.matmul(Ry, Rz))

    K = np.array([
        [W * focus / sensor_w, 0, W // 2],
        [0, H * focus / sensor_h, H // 2],
        [0, 0, 1]])
    new_K = np.array([
        [W * focus / sensor_w, 0, scale * W // 2 + X, 0],
        [0, H * focus / sensor_h, scale * H // 2 + Y, 0],
        [0, 0, 1, 0]])

    res = cv2.undistortPoints(points, K, dist, R=R, P=new_K)
    return np.reshape(res, (-1, 2)) 

def rectify_lines(config, lines):
    result = np.zeros(lines.shape)
    
    for index, points in enumerate(lines):
        result[index] = rectify_points(config, points) 

    return result


def objective_function(x, horizontal, vertical, distances):
    horizontal_rectified = rectify_lines(x, horizontal) 
    vertical_rectified = rectify_lines(x, vertical)
    distances_rectified = rectify_lines(x, distances)

    # odchylenia standardowe dla każdej linii
    horizontal_std = horizontal_rectified[:, :, 1].std(axis=1)
    vertical_std = vertical_rectified[:, :, 0].std(axis=1)

    stds = weights["std"] * (horizontal_std.sum() + vertical_std.sum())
    distance = weights["distance"] * calc_distance(distances_rectified)

    return stds + distance  
    


In [30]:
bounds = {
    'X': (-100, 100), 
    'Y': (40, 120), 
    'alpha': (-0.1, 0.1), 
    'beta': (-0.1, 0.1), 
    'gamma': (-0.1, 0.1), 
    "focus": (1.5, 6.0),
    "scale": (1.0, 1.0),
    'k1': (-0.2, 0.2),
    'k2': (-0.2, 0.2),
    'k3': (-0.2, 0.2),
    'p1': (-0.2, 0.2),
    'p2': (-0.2, 0.2)
}

init_values = {
    'X': 0, 
    'Y': 0, 
    'alpha': 0, 
    'beta': 0, 
    'gamma': 0,
    "focus": 4,
    "scale": 1,
    'k1': 0,
    'k2': 0,
    'k3': 0,
    'p1': 0,
    'p2': 0
}

x0 = np.array(list(init_values.values()))
bounds_list = list(bounds.values())

res = objective_function(x0, horizontal_points, vertical_points, distances_array)
print(res)

minimize_params = {
    "method": "nelder-mead",
    "args": (horizontal_points, vertical_points, distances_array),
    "bounds": bounds_list,
    "options": {'disp': True, "maxiter": 50000},
    # "tol": 0.1
}

res = minimize(objective_function, x0, **minimize_params)

607.5840674621836


C:\Users\wrzezniczak\AppData\Local\Temp\ipykernel_13092\1361922071.py:45: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(objective_function, x0, **minimize_params)


Optimization terminated successfully.
         Current function value: 137.926075
         Iterations: 9758
         Function evaluations: 13290


In [31]:
res

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 137.92607534579932
             x: [ 2.406e-02  6.084e+01  9.876e-02 -2.174e-02 -7.399e-02
                  1.500e+00  1.000e+00 -3.973e-02  1.494e-03 -3.202e-04
                  6.750e-04 -2.914e-05]
           nit: 9758
          nfev: 13290
 final_simplex: (array([[ 2.406e-02,  6.084e+01, ...,  6.750e-04,
                        -2.914e-05],
                       [ 2.406e-02,  6.084e+01, ...,  6.750e-04,
                        -2.914e-05],
                       ...,
                       [ 2.406e-02,  6.084e+01, ...,  6.750e-04,
                        -2.914e-05],
                       [ 2.406e-02,  6.084e+01, ...,  6.750e-04,
                        -2.914e-05]]), array([ 1.379e+02,  1.379e+02, ...,  1.379e+02,  1.379e+02]))

In [32]:
def make_config_from_res(res):
    return {
        'sensor_h': sensor_h,
        'sensor_w': sensor_w,
        'X': res.x[0],
        'Y': res.x[1],
        'alpha': res.x[2],
        'beta': res.x[3],
        'gamma': res.x[4],
        'focus': res.x[5],
        'scale': res.x[6],
        'dist': res.x[7:].tolist()
    }

rectify_config = make_config_from_res(res)
rectify_config

{'sensor_h': 3.69,
 'sensor_w': 7.38,
 'X': 0.024064762579324835,
 'Y': 60.84379344276333,
 'alpha': 0.0987555487449152,
 'beta': -0.02174072785627948,
 'gamma': -0.07399145075995611,
 'focus': 1.5000000000002707,
 'scale': 1.0,
 'dist': [-0.03972567115706957,
  0.0014939564659998909,
  -0.0003202165333903008,
  0.0006749853820165214,
  -2.914250820232034e-05]}

In [33]:
from stitch.rectify.FrameRectifier import FrameRectifier

frame_rectifier = FrameRectifier(rectify_config, W, H)

def show_rectified(b):
    frame_rectifier.calc_maps()
    image = cv2.imread(file_paths["vertical_horizontal"]["image"])
    rectified_image = frame_rectifier.rectify(image)

    rectified_image = scale_image_by_percent(rectified_image, 50)
    cv2.imshow('Frame', rectified_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

rectify_button = widgets.Button(description = "show rectified image")



display(rectify_button)

rectify_button.on_click(show_rectified)

Button(description='show rectified image', style=ButtonStyle())

## Eksport

In [34]:

export_button = widgets.Button(description = "export config")
text_field = widgets.Text(
    value='rectify_config.json',
    description='Output file:'
)

display(text_field, export_button)

def export_config(b):
    file_name = text_field.value

    with open(file_name, 'w') as f:
        json.dump(rectify_config, f)

    print(f"{file_name} saved")

export_button.on_click(export_config)

Text(value='rectify_config.json', description='Output file:')

Button(description='export config', style=ButtonStyle())